In [13]:
import numpy as np
import open3d as o3d
from pathlib import Path
from tqdm import tqdm
import cv2
from src.point_cloud_processor import load_point_cloud

def create_multi_view_images(pcd, image_size=224, n_rotations=8):
    """
    Create multi-view images from a point cloud.
    Returns a dict with keys: 'top', 'front', 'side', 'rot_0', 'rot_1', ...
    """
    if pcd is None or not pcd.has_points():
        return None

    pts = np.asarray(pcd.points)
    if pts.size == 0:
        return None

    # Center
    pts = pts - np.mean(pts, axis=0, keepdims=True)

    # Optional: downsample very dense clouds for speed
    if pts.shape[0] > 1_200_000:
        idx = np.random.choice(pts.shape[0], 1_200_000, replace=False)
        pts = pts[idx]

    def to_image(a, b):
        # Map coords to [0, image_size-1]
        amin, amax = np.min(a), np.max(a)
        bmin, bmax = np.min(b), np.max(b)
        ar = amax - amin
        br = bmax - bmin
        if ar == 0: ar = 1.0
        if br == 0: br = 1.0
        a_n = (a - amin) / ar
        b_n = (b - bmin) / br
        H, _, _ = np.histogram2d(a_n, b_n, bins=image_size, range=[[0,1],[0,1]])
        if H.max() > 0:
            H = (H / H.max()) * 255.0
        return H.astype(np.uint8)

    images = {}
    # Original orthographic projections
    images['top'] = to_image(pts[:,0], pts[:,1])    # XY
    images['front'] = to_image(pts[:,0], pts[:,2])  # XZ
    images['side'] = to_image(pts[:,1], pts[:,2])   # YZ

    # Rotated views around Z axis
    angles = np.linspace(0, 360, n_rotations, endpoint=False)
    for i, angle in enumerate(angles):
        rad = np.deg2rad(angle)
        # Rotation matrix around Z axis
        R = np.array([
            [np.cos(rad), -np.sin(rad), 0],
            [np.sin(rad),  np.cos(rad), 0],
            [0, 0, 1]
        ])
        rotated_pts = pts @ R.T
        images[f'rot_{i}'] = to_image(rotated_pts[:,0], rotated_pts[:,2])  # XZ plane for rotated side view

    return images

In [15]:
base_path = Path("..")
train_path = base_path / "train"
test_path = base_path / "test"
output_path = base_path / "data" / "multi_view_images"
output_path.mkdir(exist_ok=True)

# Processing loop
def process_and_save_images(source_root, dest_root, n_rotations=8):
    dest_root.mkdir(exist_ok=True)
    
    for species_dir in tqdm(sorted(list(source_root.iterdir())), desc=f"Processing {source_root.name}"):
        if not species_dir.is_dir():
            continue
            
        (dest_root / species_dir.name).mkdir(exist_ok=True)
        
        for file_path in species_dir.iterdir():
            if file_path.suffix in ['.pts', '.xyz', '.txt']:
                pcd = load_point_cloud(file_path)
                
                if pcd:
                    images = create_multi_view_images(pcd, n_rotations=n_rotations)
                    
                    if images:
                        base_name = file_path.stem
                        for view_name, img in images.items():
                            cv2.imwrite(str(dest_root / species_dir.name / f"{base_name}_{view_name}.png"), img)

# Run
process_and_save_images(train_path, output_path / "train", n_rotations=8)
process_and_save_images(test_path, output_path / "test", n_rotations=8)

print("Multi-view image dataset with rotated views created successfully!")


Processing test: 100%|██████████| 7/7 [00:17<00:00,  2.54s/it]

Multi-view image dataset with rotated views created successfully!
